# SAM 2023 : TP Requêtes multi-sources pour l'open data

Séances 9 et 10

Objectif : Comprendre et réaliser des jointures entre deux sources réelles distantes lorsque l'accès aux sources est conforme aux standards de l'*open data*.  


Contexte : On considère deux sources réelles ainsi que des exemples de requêtes pouvant être posées sur chacune d'entre elles.

On vous demande de mettre en oeuvre des scénarios de requêtes **MULTI** sources. 




## Préparation

**Copier** ce notebook dans votre drive et l'exécuter dans l'environnement colab de google


### Manipuler le résultat d'une requête
Fonctions **tableauResultat** pour extraire  le résultat d'une requête

In [2]:
import pandas as pd

#pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Déterminer la liste des noms d'attributs du résultat, en tenant compte des attributs optionnels.
def schemaResultat(resultats, limit=100):
  d = {}
  for elt in resultats[:limit]:
    for (pos, nom) in enumerate(elt):
      sum_pos, nb = d.get(nom, (0, 0))
      d[nom] = (sum_pos + pos, nb + 1)
  mean_pos_by_name = [(sum_pos / nb, nom) for (nom, (sum_pos, nb)) in d.items()]
  colonnes = [nom for (mean_pos, nom) in sorted(mean_pos_by_name)]
  return colonnes

def tableauResultat(resultats, limit=100, colonnes=None):
  if colonnes is None :
    colonnes = schemaResultat(resultats, limit)
  tableau = []
  for elt in resultats[:limit]:
    row = {name:v.value for name, v in elt.items()}
    tableau.append([row.get(n, "") for n in colonnes])
  return tableau

print ("fonctions définies")

fonctions définies


Afficher le résultat

In [3]:
def afficherResultat(resultats, limit=100):
  print(len(resultats), "éléments")
  colonnes = schemaResultat(resultats, limit)
  print("schéma: ", colonnes)
  result = tableauResultat(resultats, limit, colonnes)
  return pd.DataFrame(result, columns = colonnes)

print ("fonctions définies")

fonctions définies


### Importer le pilote pour accéder aux sources

In [4]:
from SPARQLWrapper import SPARQLWrapper2
print("pilote importé")

pilote importé


### Préparer l'accès à la source BNF

In [5]:
# Connexion à la BNF
BNF = SPARQLWrapper2("http://data.bnf.fr/sparql")

BNF_prefix = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rda: <http://rdaregistry.info/Elements/w/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX bnf: <http://data.bnf.fr/ontology/bnf-onto/>

PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX voc2: <http://rdvocab.info/ElementsGr2/>
"""
# print(BNF_prefix)

### Préparer l'accès à la source DBPEDIA

In [6]:
# Connexion à DBPEDIA France. 
DBPEDIA = SPARQLWrapper2("http://fr.dbpedia.org/sparql")

DBPEDIA_prefix =  """
PREFIX dc:   <http://purl.org/dc/terms/>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX dbo:  <http://dbpedia.org/ontology/>

# Dans cette source, les noms de propriétés sont en français
PREFIX dbp:  <http://fr.dbpedia.org/property/>
"""
# print(DBPEDIA_prefix)

## Source 1 : La bibliothèque BNF

### BNF P1: Le nom des personnes
On suppose les relations : 
*    **CrééPar**(ouvrage, auteur)
*    **Nom**(auteur, nom)

En SQL, la requête s'exprimerait ainsi :

```sql
select distinct n.nom
from CrééPar c, Nom n
where c.auteur = n.auteur
```

In [9]:
requete = BNF_prefix + """
SELECT distinct ?nom
WHERE {
  ?ouvrage dc:creator ?auteur .       # un ouvrage est créé par au moins un auteur
  ?auteur foaf:name ?nom              # un auteur a un nom
}
LIMIT 5
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
for r in tableauResultat(resultats):
  print(r)

['Fernand Gilles Lagrange']
['Musée des beaux-arts. Rouen']
['Château-Musée. Dieppe, Seine-Maritime']
['Évelyne Brisou-Pellen']
['Philippe Delestre']


### BNF T1: liste de thèmes
#### Les *themes* des oeuvres dont l'auteur est référencé dans dbpedia france

En SQL, cette requête pourrait s'exprimer sur les tables suivantes

**créé_par** (livre, auteur)

**type_d_expression** (livre, type_expre)  -- exple de tpye d'expression: textuelle

**écrit_en** (livre, langue)

**sujet** (livre, thème)

**auteur_équivalent** (auteur, auteur_source2)

et une condition SQL serait : WHERE auteur_source2 like 'http://fr.dbpedia.org%'

In [16]:
requete = BNF_prefix + """
SELECT distinct ?theme
WHERE {
  
  # une oeuvre est associée a (au moins) un auteur
  ?livre dc:creator ?auteur .

  # un livre est un ouvrage dont la forme d'expression est textuelle (te)
  # ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .
  
  # livre écrit en langue française
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a au moins un theme
  ?livre bnf:subject ?theme.

  # l'auteur existe également dans une autre source,
  # on dit qu'il est "aligné" avec un autre instance du même individu.
  ?auteur owl:sameAs ?auteurDbpedia .

  # la référence externe "pointe" vers dbpedia
  FILTER ( regex(?auteurDbpedia, "http://fr.dbpedia.org")  )
 
}
ORDER BY ?theme
LIMIT 20
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)

# Exemple:
# Informatique
# Langues
# Philosophie

20 éléments
schéma:  ['theme']


,theme
0,Administration publique
1,Anthropologie. Ethnologie
2,Architecture
3,Archéologie. Préhistoire. Histoire ancienne
4,Art
5,Art et science militaires
6,Arts du spectacle
7,Arts graphiques
8,Astronomie
9,Audiovisuel


###  BNF A1 : une liste d'auteurs 

A1 : Liste des auteurs de livres écrits en français et nés pendant une certaine période, par exemple au 20eme siècle. Afficher le nom de l'auteur, son année et lieu de naissance.



In [11]:
requete = BNF_prefix + """
SELECT distinct ?nom  ?naissance ?lieu
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom 
?auteur foaf:name ?nom .

# un auteur a une année et un lieu de naissance
?auteur bio:birth ?naissance.
?auteur voc2:placeOfBirth ?lieu

# on sélectionne les années de naissance dans l'intervalle [1900, 2000]
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer 
     && xsd:integer (?naissance) < '2000'^^xsd:integer )
}
#ORDER BY ?nom
LIMIT 20
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)

20 éléments
schéma:  ['nom', 'naissance', 'lieu']


,nom,naissance,lieu
0,Annie Rey-Goldzeiguer,1925,Tunis (Tunisie)
1,David Cohen,1922,Tunis
2,André Chardine,1902,Fécamp (Seine-Maritime)
3,Christian Morissonneau,1939,Cholet (Maine-et-Loire)
4,Jean Ferrandi,1920,Muro (Haute-Corse)
5,Jacques Soisson,1928,Paris
6,Abderrahman Slaoui,1919,"Fès, Maroc"
7,Jacques Poinson,1939,Ambert (Puy-de-Dôme)
8,Diane Canivet,1919,"Alexandrie, Egypte"
9,René Coustellier,1920,Port-Saint-Louis (Bouches-du-Rhône)


###  BNF A2 : les auteurs de la BNF référencés dans dbpedia
Les auteurs ayant un identifiant externe pointant vers dbpedia


In [12]:
requete = BNF_prefix + """
SELECT distinct ?nom  ?naissance ?auteurDbpedia
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom, et une année de naissance
?auteur foaf:name ?nom ; bio:birth ?naissance .

# l'auteur existe également dans une source externe,
# on dit qu'il est "aligné" avec un autre instance du même individu.
?auteur owl:sameAs ?auteurDbpedia .

# naissance dans l'intervalle [1900, 2000] et la référence externe "pointe" vers dbpedia
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer 
     && xsd:integer (?naissance) < '2000'^^xsd:integer 
     && regex(?auteurDbpedia, "http://fr.dbpedia.org") 
     )
}
ORDER BY desc(?naissance)
LIMIT 10
""" 

BNF.setQuery(requete)

resultats = BNF.query().bindings
afficherResultat(resultats)

10 éléments
schéma:  ['nom', 'naissance', 'auteurDbpedia']


,nom,naissance,auteurDbpedia
0,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
1,Isabelle Périer,1978,http://fr.dbpedia.org/resource/Isabelle_P%C3%A9rier
2,Nathanaël Dupré La Tour,1977,http://fr.dbpedia.org/resource/Nathana%C3%ABl_Dupr%C3%A9_La_Tour
3,Coralie Delaume,1976,http://fr.dbpedia.org/resource/Coralie_Delaume
4,Claire Guézengar,1972,http://fr.dbpedia.org/resource/Claire_Guezengar
5,Philippe Nassif,1971,http://fr.dbpedia.org/resource/Philippe_Nassif
6,Benoït Violier,1971,http://fr.dbpedia.org/resource/Beno%C3%AEt_Violier
7,Pierre Cherruau,1969,http://fr.dbpedia.org/resource/Pierre_Cherruau
8,Emmanuelle Marie,1965,http://fr.dbpedia.org/resource/Emmanuelle_Marie
9,Thierry Wanegffelen,1965,http://fr.dbpedia.org/resource/Thierry_Wanegffelen


### BNF A3 : Les auteurs d'une oeuvre philosophique

Voir T1 pour la liste des thèmes possibles autres que la philosophie.

On se limite aux oeuvres écrite en français et sont l'auteur est référencé dans dbpedia france.

In [15]:
requete = BNF_prefix + """
SELECT distinct ?nom ?auteurDbpedia 
WHERE { 
  
  ?livre dc:creator ?auteur .

  # un livre est un ouvrage textuel
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a pour theme "Philosophie"
  ?livre bnf:subject "Philosophie" .

  # le nom de l'auteur
  ?auteur foaf:name ?nom .

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER ( regex(?livreWiki, "http://wikidata.org") 
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?auteurDbpedia

LIMIT 20
"""

BNF.setQuery(requete)
resultats =  BNF.query().bindings
afficherResultat(resultats, 5)

0 éléments
schéma:  []


""


###  BNF L1 : Les livres avec leurs auteurs

Les livres écrit en français au 21eme siècle,  référencés dans wikidata et dont l'auteur est référencé dans dbpedia

In [14]:
requete = BNF_prefix + """
SELECT distinct ?annee ?nom ?titre ?auteurDbpedia
WHERE { 
  
  ?livre dc:creator ?auteur .

  # l'année de parution du livre
  ?livre dc:date ?annee .

  # le titre du livre
  ?livre dc:title ?titre .

  # un livre est un ouvrage textuel
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le nom de l'auteur
  ?auteur foaf:name ?nom.

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER (
       xsd:integer (?annee) > '2000'^^xsd:integer 
    && xsd:integer (?annee) < '2099'^^xsd:integer 
    && regex(?livreWiki, "http://wikidata.org") 
    &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
#ORDER BY desc(?annee)

LIMIT 20
"""

BNF.setQuery(requete)
resultats =  BNF.query().bindings
afficherResultat(resultats)

0 éléments
schéma:  []


""


## Source 2 : DBPedia France

### D1 : Informations sur une personne dont le nom contient certains mots.
Afficher le nom, nationalité, profession, date et lieu de naissance

In [ ]:
# l'expression régulière du nom à retrouver
MOTIF = "Victor.*Hug"

requete_parametree = DBPEDIA_prefix + """
  SELECT ?p ?name ?nationalité ?profession year(xsd:dateTime(?naissance)) as ?naissance  ?lieu
  WHERE { 
    ?p a dbo:Person .
    ?p foaf:name ?name 
    optional {?p dbp:nationalité ?nationalité}
    optional {?p dbp:profession ?profession}
    optional { ?p dbo:birthDate ?naissance}
    optional {?p dbp:lieuDeNaissance ?lieu}

  FILTER (regex(?p, "{MOTIF}", "i") )
  }
  LIMIT 10
"""

requete = requete_parametree.replace("{MOTIF}", MOTIF)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

5 éléments
schéma:  ['p', 'name', 'nationalitÃ©', 'profession', 'naissance', 'lieu']


,p,name,nationalitÃ©,profession,naissance,lieu
0,http://fr.dbpedia.org/resource/Victor_Hugues,Victor Hugues,,,1762,à Marseille
1,http://fr.dbpedia.org/resource/Victor_Hugo,Victor Hugo,http://fr.dbpedia.org/resource/France,http://fr.dbpedia.org/resource/Écrivain,1802,http://fr.dbpedia.org/resource/Maison_natale_de_Victor_Hugo
2,http://fr.dbpedia.org/resource/Victor_Hugo,Victor Hugo,http://fr.dbpedia.org/resource/France,http://fr.dbpedia.org/resource/Écrivain,1802,http://fr.dbpedia.org/resource/Besançon
3,http://fr.dbpedia.org/resource/Victor_Hugo__1,Victor Hugo,,,1802,
4,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao


### D2 : les infos détaillées sur une certaine personne

In [ ]:
# On fixe une certaine personne
PERSONNE = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#PERSONNE = 'http://fr.dbpedia.org/resource/Jean-Paul_Sartre'

# substituer une variable par sa valeur dans une chaine de caractères
#id = f"<{PERSONNE}>"
id = "<{PERSONNE}>".replace("{PERSONNE}", PERSONNE)
print("l'identifiant complet de la personne recherchée est :", id)

requete_parametree = DBPEDIA_prefix + """
  SELECT distinct ?name ?nationalite ?interet ?idee
  WHERE { 
    ?pers foaf:name ?name 
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

#      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
#      optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> ) 
  }
  order by ?p
  LIMIT 20
"""

# fixer le nom de la personne dans la requête 
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

# print("la requête est :", requete)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

l'identifiant complet de la personne recherchée est : <http://fr.dbpedia.org/resource/Simone_de_Beauvoir>
3 éléments
schéma:  ['name', 'nationalite', 'interet', 'idee']


,name,nationalite,interet,idee
0,Simone de Beauvoir,Française,http://fr.dbpedia.org/resource/Féminisme,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
1,Simone de Beauvoir,Française,http://fr.dbpedia.org/resource/Politique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
2,Simone de Beauvoir,Française,http://fr.dbpedia.org/resource/Éthique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"


### D3 : la liste des oeuvres principales d'une personne donnée

In [ ]:
# On fixe une certaine personne
PERSONNE = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'

requete_parametree = DBPEDIA_prefix +  """
    SELECT distinct ?name ?oeuvre
    WHERE { 
      ?pers foaf:name ?name 
      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
#     optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> ) 
}
order by ?p
LIMIT 20
"""
# fixer le nom de la personne dans la requête 
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

4 éléments
schéma:  ['name', 'oeuvre']


,name,oeuvre
0,Simone de Beauvoir,L'Invitée
1,Simone de Beauvoir,Le Deuxième Sexe
2,Simone de Beauvoir,Les Belles Images
3,Simone de Beauvoir,Les Mandarins


### D4 : les influenceurs d'une personne donnée

In [ ]:
# On fixe une certaine personne
PERSONNE = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#PERSONNE = 'http://fr.dbpedia.org/resource/Judith_Butler'

requete_parametree = DBPEDIA_prefix + """
   SELECT distinct ?name ?influenceur
    WHERE { 
      ?pers foaf:name ?name 
      optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> ) 
}
order by ?p
LIMIT 20
"""

# fixer le nom de la personne dans la requête 
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
print(len(res), 'resultats')
afficherResultat(res)

11 resultats
11 éléments
schéma:  ['name', 'influenceur']


,name,influenceur
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Jean-Paul_Sartre
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Engels
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Mary_Wollstonecraft
3,Simone de Beauvoir,http://fr.dbpedia.org/resource/Edmund_Husserl
4,Simone de Beauvoir,http://fr.dbpedia.org/resource/Emmanuel_Kant
5,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Nietzsche
6,Simone de Beauvoir,http://fr.dbpedia.org/resource/Georg_Wilhelm_Friedrich_Hegel
7,Simone de Beauvoir,http://fr.dbpedia.org/resource/Karl_Marx
8,Simone de Beauvoir,http://fr.dbpedia.org/resource/Martin_Heidegger
9,Simone de Beauvoir,http://fr.dbpedia.org/resource/René_Descartes


### D5 : Les auteurs nés en 1950
Utiliser la fonction year pour extraire l'année de naissance à partir d'une date de naissance.

In [ ]:
# On fixe une certaine année
ANNEE = 1950

requete_parametree = DBPEDIA_prefix + """
    SELECT ?p ?name year(xsd:dateTime(?naissance)) as ?naissance
    WHERE { 
      ?p a dbo:Person .
      ?p foaf:name ?name .
      ?p dbo:birthDate ?naissance
      FILTER( year(xsd:dateTime(?naissance)) = {ANNEE})
    }
order by ?p
LIMIT 10
"""

requete = requete_parametree.replace("{ANNEE}", str(ANNEE))

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['p', 'name', 'naissance']


,p,name,naissance
0,http://fr.dbpedia.org/resource/Abbas_Babai,Abbas Babai,1950
1,http://fr.dbpedia.org/resource/Abbas_Doran,Abbas Doran,1950
2,http://fr.dbpedia.org/resource/Abdelali_Boughrara,Abdelali Boughrara,1950
3,http://fr.dbpedia.org/resource/Abdelkrim_Zbidi,Abdelkrim Zbidi,1950
4,http://fr.dbpedia.org/resource/Abderraouf_Ayadi,Abderraouf Ayadi,1950
5,http://fr.dbpedia.org/resource/Abderrazak_Zouari,Abderrazak Zouari,1950
6,http://fr.dbpedia.org/resource/Abdullah_Gül,Abdullah Gül,1950
7,http://fr.dbpedia.org/resource/Abraham_Wald,Abraham Wald,1950
8,http://fr.dbpedia.org/resource/Adrian_Năstase,Adrian Năstase,1950
9,http://fr.dbpedia.org/resource/Adriano_Panatta,Adriano Panatta,1950


### D6 : La liste de tous les intérets principaux 


In [ ]:
requete = DBPEDIA_prefix + """
    SELECT distinct ?interet
    WHERE { 
      ?personne dbo:mainInterest ?o .
      ?o rdfs:label ?interet .

      FILTER (langMatches( lang(?interet), "FR" ) )
}
order by ?interet
LIMIT 50
"""

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

50 éléments
schéma:  ['interet']


,interet
0,Activisme politique
1,Agnosticisme
2,Alchimie
3,Amour
4,Analyse littéraire
5,Angélologie
6,Anthropologie
7,Anthropologie politique
8,Apologétique
9,Aqida


### D7 la liste des philosophes dans Dbpedia

In [ ]:
import time

requete = DBPEDIA_prefix + """
    
SELECT distinct  ?name ?p
WHERE { 
  ?p rdf:type dbo:Philosopher .
  ?p foaf:name ?name
}
order by ?name
limit 10
"""

start_time = time.time()

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings

duree = int ( (time.time() - start_time) * 1000)
print(f"durée {duree} ms")

afficherResultat(res)


durée 212 ms
10 éléments
schéma:  ['name', 'p']


,name,p
0,(),http://fr.dbpedia.org/resource/Mélissos
1,(Cheikh Mustafâ 'Abd al-Azîz),http://fr.dbpedia.org/resource/Michel_Vâlsan
2,(née de Lestrange),http://fr.dbpedia.org/resource/Monique_Gessain
3,(penseur allemand),http://fr.dbpedia.org/resource/Bruno_Bauer
4,(’Abd al-Wâhid Yahyâ),http://fr.dbpedia.org/resource/René_Guénon
5,Abd ar-Rahman ibn Nasir as-Sadi au-Tamimi,http://fr.dbpedia.org/resource/Abd_ar-Rahman_ibn_Nasir_as-Sadi
6,Abdennour Bidar,http://fr.dbpedia.org/resource/Abdennour_Bidar
7,Abou Ḥamid Moḥammed ibn Moḥammed al-Ghazālī,http://fr.dbpedia.org/resource/Al-Ghazâlî
8,Adam Smith,http://fr.dbpedia.org/resource/Adam_Smith
9,African Spir,http://fr.dbpedia.org/resource/African_Spir


# Exercice 1 : Requêtes réparties entre les 2 sources BNF et DBPedia
A compléter

### J1: Jointure A3 avec D2 par boucles imbriquées
Proposer une solution pour calculer la jointure entre A3 sur le site BNF et D2 sur le site DBPedia.
Pour chaque livre de afficher son titre et son année, le nom de l'auteur, sa nationalité et la liste de ses idées remarquables.

In [20]:
requete = BNF_prefix + """
SELECT distinct ?nom ?auteurDbpedia 
WHERE { 
  
  ?livre dc:creator ?auteur .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a pour theme "Philosophie"
  ?livre bnf:subject "Philosophie" .

  # le nom de l'auteur
  ?auteur foaf:name ?nom .

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER ( regex(?livreWiki, "http://wikidata.org") 
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?auteurDbpedia
"""

BNF.setQuery(requete)
res1 =  BNF.query().bindings
afficherResultat(resultats, 5)

20 éléments
schéma:  ['nom', 'auteurDbpedia']


,nom,auteurDbpedia
0,Alexis Carrel,http://fr.dbpedia.org/resource/Alexis_Carrel
1,Antoine de Rivarol,http://fr.dbpedia.org/resource/Antoine_de_Rivarol
2,Bernard-Henri Lévy,http://fr.dbpedia.org/resource/Bernard-Henri_L%C3%A9vy
3,Bernard de Fontenelle,http://fr.dbpedia.org/resource/Bernard_Le_Bouyer_de_Fontenelle
4,Claude Guillon,http://fr.dbpedia.org/resource/Claude_Guillon_%28%C3%A9crivain%29


In [21]:
# On fixe une certaine personne
PERSONNE = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#PERSONNE = 'http://fr.dbpedia.org/resource/Jean-Paul_Sartre'

# substituer une variable par sa valeur dans une chaine de caractères
#id = f"<{PERSONNE}>"
id = "<{PERSONNE}>".replace("{PERSONNE}", PERSONNE)
print("l'identifiant complet de la personne recherchée est :", id)

requete_parametree = DBPEDIA_prefix + """
  SELECT distinct ?name ?nationalite ?interet ?idee
  WHERE { 
    ?pers foaf:name ?name 
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

#      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
#      optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> ) 
  }
  order by ?p
"""

# fixer le nom de la personne dans la requête 
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

# print("la requête est :", requete)

DBPEDIA.setQuery(requete)
res2 = DBPEDIA.query().bindings
afficherResultat(res)

l'identifiant complet de la personne recherchée est : <http://fr.dbpedia.org/resource/Simone_de_Beauvoir>
6 éléments
schéma:  ['name', 'interet', 'idee']


,name,interet,idee
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Philosophie,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Politique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Éthique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
3,,http://fr.dbpedia.org/resource/Philosophie,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
4,,http://fr.dbpedia.org/resource/Politique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
5,,http://fr.dbpedia.org/resource/Éthique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"


In [24]:
colonnes = schemaResultat(res2)
tableau = []

for r1 in res1:
    for r2 in res2:
        if r1["nom"] == r2["name"]:
            row = {name:v.value for name, v in r2.items()}
            tableau.append([row.get(n, "") for n in colonnes])

tableau

[]

### J2 : Pour une année N afficher la liste des personnes qui sont nées l'année N et qui existent à la fois dans la BNF et dans DBPedia.
Proposer une solution par fusion de deux résultats triés

In [ ]:
# A compléter

### J3 : Quels sont les auteurs d'une oeuvre dont le thème est philosophie (dans la BNF) et qui sont référencés comme étant des philosophes dans DBpedia ?

In [ ]:
# A compléter

# Exercice 2 : Accès aux données par des requêtes de sélections
L'objectif de cet exercice est d'appréhender la situation où l'accès aux données est contraint (voir la notion de [linked data fragment](https://linkeddatafragments.org/concept/)). Le point d'accès n'autorise que des requêtes simples (donc peut coûteuses à traiter) qui ne contiennent que des **sélections**. La clause where d'une requête ne peut contenir qu'**un seul** motif ayant 1 ou 2 variables. Les motifs autorisés sont donc parmi :
*   ?a *propriété* ?b *(cette requête a deux variables)*
*   *A* *propriété* ?b  *(cette requête a une seule variable)*
*   ?a *propriété* *B*  

où ?a et ?b sont des variables et *A*, *B* et *propriété* sont des valeurs fixées avant de poser la requête.

Ainsi, toutes les jointures doivent être exprimées par le client qui accède aux données : vous devez "programmer" toutes les jointures en python.
Pour les requêtes D1 et D6 proposer une solution respectant ce mode d'accès.

In [ ]:
# à compléter

# Exercice 3 : Accès par morceau au résultat d'une requête
1) Un utilisateur veut accéder aux données de manière progressive. Proposer une solution pour obtenir le resultat d'une requete par morceau de N tuples, en utilisant les mots-clés OFFSET et LIMIT. Appliquer cela pour lire le résultat de la requête qui affiche le nom des personnes existant dans dbpedia.
Etudier l'influence de l'offset et de la taille du résultat sur le temps de réponse de la requête.



In [ ]:
# Exemple d'utilisation de OFFSET et LIMIT

# Compléter cet exemple


import time

requete = DBPEDIA_prefix + """

SELECT distinct ?name ?p 
WHERE { 
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
}
offset 33000 limit 10
"""

start_time = time.time()
# exécution de la requête
DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
duree = int ( (time.time() - start_time) * 1000)
print(f"la requête dure {duree} ms")

afficherResultat(res)

la requête dure 394 ms
10 éléments
schéma:  ['name', 'p']


,name,p
0,Éric et Ramzy,http://fr.dbpedia.org/resource/Éric_et_Ramzy
1,Étienne Eustache Bruix,http://fr.dbpedia.org/resource/Étienne_Eustache_Bruix
2,Étienne Lenoir,http://fr.dbpedia.org/resource/Étienne_Lenoir
3,50 Cent,http://fr.dbpedia.org/resource/50_Cent
4,Abbas,http://fr.dbpedia.org/resource/Abbas_Ier_Hilmi
5,عباس حلمي الأول,http://fr.dbpedia.org/resource/Abbas_Ier_Hilmi
6,Abbas,http://fr.dbpedia.org/resource/Abbas_Ier_le_Grand
7,( عبد الله الثانى),http://fr.dbpedia.org/resource/Abdallah_II
8,Abdallah II,http://fr.dbpedia.org/resource/Abdallah_II
9,Abdelatif Benazzi,http://fr.dbpedia.org/resource/Abdelatif_Benazzi


2) Définir une fonction traiteRequete(requete) qui retourne un generateur sur une requete de telle sorte que la requete soit lue par morceaux de N tuples et que seuls les morceaux permettant de produire les M lignes demandées par l'utlisateur soit lus. On ne connait pas à l'avance le nombre M de lignes que va lire l'utilisateur. On a M > N.
Indication voir le mot clé python **yield** pour retourner un générateur.

In [ ]:
requete = DBPEDIA_prefix + """
SELECT distinct ?name ?p 
WHERE { 
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
}
offset 0 limit 10
"""

# définition d'un générateur
def traiteRequete(requete):
  DBPEDIA.setQuery(requete)
  res = DBPEDIA.query().bindings
  print(f"leture d'un morceau de {len(res)} lignes")
  for r in res :
    yield r


# Nombre de tuples que l'utilisateur souhaite lire
a_lire = 20

lu = 0
for a in traiteRequete(requete):
  print(a)
  lu += 1
  if(lu >= a_lire):
    break

print()
if(lu < a_lire):
  print(f"Attention !!!! l'utilisateur n'a obtenu que {lu} lignes sur les {a_lire}")
else:
  print(f"OK, l'utilisateur a obtenu {lu} lignes sur les {a_lire}")

leture d'un morceau de 10 lignes
{'name': Value(literal:'Olsen'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Aaron_Olsen')}
{'name': Value(literal:'Aaron Rouge-Serret'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Aaron_Rouge-Serret')}
{'name': Value(literal:'Aaron Yoo'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Aaron_Yoo')}
{'name': Value(literal:'Abdelkader Bakhtache'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Abdelkader_Bakhtache')}
{'name': Value(literal:'Abderrahim Bouabid'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Abderrahim_Bouabid')}
{'name': Value(literal:'CheikhAbdul Sattar Abou Richa (:  عبد الستار أبو ريشة)'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Abdul_Sattar_Abou_Richa')}
{'name': Value(literal:'Abe Segal'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Abe_Segal')}
{'name': Value(literal:'Abel Marie Decaux'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Abel_Decaux')}
{'name': Value(literal:'Abel Matutes'), 'p': Value(uri:'http://fr.

3) Proposer une solution pour accéder de manière progressive au résultat d'une requête répartie entre 2 sites, par exemple J1.

In [ ]:
# à compléter

# Exerice 4 : Cache de requêtes
1) Proposer des améliorations des requêtes précédentes afin d'éviter de poser plusieurs fois une même requête. On suppose que le résultat de la requête ne change pas entre deux invocations de la même requête.

2) Cache avec une capacité limitée : considérer qu'on peut conserver en cache le résultat de C requêtes (C=10) au maximum

In [ ]:
# A compléter

# FAQ: Questions diverses


## Mise au point des requêtes
Pour débugguer et **mettre au point vos requêtes** Sparql, il est utile de les tester directement sur l'interface du point d'accès dédié de la source de données. L'avantage est que cela vous retourne un message indiquant très précisément la nature et l'endroit où se trouve l'erreur dans la requête.
*   [BNF](http://data.bnf.fr/sparql) voir des [exemples](https://api.bnf.fr/sparql-endpoint-de-databnffr)
*   [DBPedia](http://fr.dbpedia.org/sparql) et la liste de tous les [prefixes](http://fr.dbpedia.org/sparql?nsdecl) 


## Autres sources de données
Autres sources à votre disposition pour proposer des requêtes multi-sources:

1.   [Mondial](http://www.semwebtech.org/mondial/10/sparql) : données géopolitiques. Attention le résulat des requêtes est en XML pas en JSON.
2.   [Persée](data.persee.fr) : données bibliographiques alignées avec plusieurs autres sources dont dbpedia. Voir des [exemples](http://data.persee.fr/ressources/requetes-exemples/)
3.   [BioPortal](http://sparql.bioontology.org)
4.    [IdRef](https://data.idref.fr/sparql) serveur d’autorité IdRef et les références bibliographiques en provenance du Sudoc. 
5.   [Insee](https://rdf.insee.fr/sparql)
6.  [Geonames web services](https://www.geonames.org/export/ws-overview.html)


## SPARKLIS : aide à l'écriture de requêtes Sparql
Pour exprimer vos requêtes en *langage naturel* et les traduire automatiquement en SPARQL, voir l'appli [Sparklis](http://www.irisa.fr/LIS/ferre/sparklis/)


*   onglet **Sparklis** pour exprimer vos requêtes en langage naturel
*   onglet **YASGUI** pour voir la traduction en Sparql



## Nombre d'éléments dans le resultat d'une requête
Pour compter le nombre d'éléments d'une requête, utiliser la notation avec requête imbriquée dans le where :

SELECT (count(1) as ?nb)

WHERE { *sous-requete* }
